<a href="https://colab.research.google.com/github/kamranr123/kamranr123.github.io/blob/master/fine_tune_multilingual_bert_on_emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Install dependencies
!pip install -q transformers datasets evaluate accelerate

In [ ]:
!wget -P /content/drive/MyDrive/AI/emotions_classifier https://github.com/nazaninsbr/Persian-Emotion-Detection/raw/refs/heads/main/dataset.csv

In [1]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
import evaluate

# Disable W&B
os.environ["WANDB_DISABLED"] = "true"


In [36]:
# Load CSV
df = pd.read_csv("/content/drive/MyDrive/AI/emotions_classifier/dataset.csv")

# Emotion label columns
label_cols = ["Anger", "Fear", "Happiness", "Hatred", "Sadness", "Wonder"]

# Convert vote counts -> binary labels (≥3 = 1)
df[label_cols] = df[label_cols].map(lambda x: 1 if x >= 2 else 0)

# Add Neutral label
df["Neutral"] = df[label_cols].apply(lambda row: 1 if row.sum() == 0 else 0, axis=1)

# Final label list
final_labels = label_cols + ["Neutral"]
num_labels = len(final_labels)

# Pack labels into list and convert to float
df["labels"] = df[final_labels].apply(lambda row: [float(x) for x in row], axis=1)

# Keep only text + labels
df = df[["text", "labels"]]

# Train/validation split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Create Hugging Face datasets
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))

dataset = DatasetDict({"train": train_ds, "validation": val_ds})

# Optional: set torch format
dataset.set_format(type="torch", columns=["text", "labels"])

# # for test
# small_train_ds = dataset["train"].select(range(min(1024, len(dataset["train"]))))
# small_val_ds   = dataset["validation"].select(range(min(1024, len(dataset["validation"]))))

# # Create a small dataset dict
# small_dataset = DatasetDict({
#     "train": small_train_ds,
#     "validation": small_val_ds
# })

print(f"Number of labels: {num_labels}")
print(f"Classes: {final_labels}")


Number of labels: 7
Classes: ['Anger', 'Fear', 'Happiness', 'Hatred', 'Sadness', 'Wonder', 'Neutral']


In [37]:
model_name = "google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.remove_columns(["text"])

# small_dataset = small_dataset.map(tokenize, batched=True)
# small_dataset = small_dataset.remove_columns(["text"])

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    problem_type="multi_label_classification"
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    preds = (probs > 0.5).int().numpy()
    labels = labels.astype(int)

    f1 = f1_score(labels, preds, average="macro")
    # subset accuracy
    acc = (preds == labels).all(axis=1).mean()
    return {"accuracy": acc, "f1_macro": f1}


In [25]:
args = TrainingArguments(
    output_dir="/content/emotions_classifier/bert-persian-emotions",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,  # effective batch = 16
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True,                     # mixed precision
    report_to="none",                # disables W&B
    optim="adamw_torch_fused"           # faster optimizer if CUDA 11.7+
)


In [32]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipython-input-2859974727.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.216000,0.204014,0.804833,0.127409
2,0.214500,0.203465,0.804833,0.127409
3,0.209100,0.203035,0.804833,0.127409


TrainOutput(global_step=4500, training_loss=0.21229507615831164, metrics={'train_runtime': 863.0885, 'train_samples_per_second': 83.421, 'train_steps_per_second': 5.214, 'total_flos': 4736211609600000.0, 'train_loss': 0.21229507615831164, 'epoch': 3.0})

In [ ]:
texts = [
    "من امروز خیلی خوشحالم",      # Happy
    "احساس می‌کنم ناراحت و خسته‌ام", # Sad
    "از تاریکی می‌ترسم",           # Fear
    "قدم زدن زیر بارون شاید بهترین مسکن درد هاست..." # Neutral
]

inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
outputs = model(**inputs)
probs = torch.sigmoid(outputs.logits).detach().numpy()

for text, p in zip(texts, probs):
    labels_pred = [final_labels[i] for i, v in enumerate(p) if v > 0.5]
    print(text, "->", labels_pred)


In [34]:
import numpy as np
from sklearn.metrics import f1_score, classification_report

# collect true labels and preds on validation set
preds_logits = trainer.predict(dataset["validation"]).predictions  # raw logits
probs = 1 / (1 + np.exp(-preds_logits))
preds = (probs > 0.5).astype(int)

refs = np.stack([ex["labels"] for ex in dataset["validation"]])  # shape (N, num_labels)

# overall per-label counts
pos_counts = refs.sum(axis=0)
neg_counts = refs.shape[0] - pos_counts
print("pos counts per label:", pos_counts)
print("neg counts per label:", neg_counts)

# per-label f1
for i, name in enumerate(final_labels):   # final_labels from your prep
    print(name, "F1:", f1_score(refs[:, i], preds[:, i], zero_division=0))

# full classification report (flattened per-label)
print(classification_report(refs, preds, zero_division=0))


pos counts per label: [ 295.  136.  125.  245.  356.  179. 4829.]
neg counts per label: [5705. 5864. 5875. 5755. 5644. 5821. 1171.]
Anger F1: 0.0
Fear F1: 0.0
Happiness F1: 0.0
Hatred F1: 0.0
Sadness F1: 0.0
Wonder F1: 0.0
Neutral F1: 0.8918644380829255
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       295
           1       0.00      0.00      0.00       136
           2       0.00      0.00      0.00       125
           3       0.00      0.00      0.00       245
           4       0.00      0.00      0.00       356
           5       0.00      0.00      0.00       179
           6       0.80      1.00      0.89      4829

   micro avg       0.80      0.78      0.79      6165
   macro avg       0.11      0.14      0.13      6165
weighted avg       0.63      0.78      0.70      6165
 samples avg       0.80      0.80      0.80      6165



In [35]:
import numpy as np

# Convert Hugging Face dataset into numpy array of labels
all_labels = np.stack(dataset["train"]["labels"])  # shape: (num_samples, num_labels)

# Count positives and negatives per label
pos_counts = all_labels.sum(axis=0)
neg_counts = all_labels.shape[0] - pos_counts

print("pos counts per label:", pos_counts)
print("neg counts per label:", neg_counts)

# If you want mapping to label names:
for name, pos, neg in zip(final_labels, pos_counts, neg_counts):
    print(f"{name:10s} | pos: {int(pos):5d} | neg: {int(neg):5d}")


pos counts per label: [ 1337.   554.   567.  1011.  1414.   807. 19107.]
neg counts per label: [22663. 23446. 23433. 22989. 22586. 23193.  4893.]
Anger      | pos:  1337 | neg: 22663
Fear       | pos:   554 | neg: 23446
Happiness  | pos:   567 | neg: 23433
Hatred     | pos:  1011 | neg: 22989
Sadness    | pos:  1414 | neg: 22586
Wonder     | pos:   807 | neg: 23193
Neutral    | pos: 19107 | neg:  4893


In [38]:
import numpy as np

# Convert Hugging Face dataset into numpy array of labels
all_labels = np.stack(dataset["train"]["labels"])  # shape: (num_samples, num_labels)

# Count positives and negatives per label
pos_counts = all_labels.sum(axis=0)
neg_counts = all_labels.shape[0] - pos_counts

print("pos counts per label:", pos_counts)
print("neg counts per label:", neg_counts)

# If you want mapping to label names:
for name, pos, neg in zip(final_labels, pos_counts, neg_counts):
    print(f"{name:10s} | pos: {int(pos):5d} | neg: {int(neg):5d}")


pos counts per label: [5363. 3394. 3155. 4775. 5797. 4121. 7864.]
neg counts per label: [18637. 20606. 20845. 19225. 18203. 19879. 16136.]
Anger      | pos:  5363 | neg: 18637
Fear       | pos:  3394 | neg: 20606
Happiness  | pos:  3155 | neg: 20845
Hatred     | pos:  4775 | neg: 19225
Sadness    | pos:  5797 | neg: 18203
Wonder     | pos:  4121 | neg: 19879
Neutral    | pos:  7864 | neg: 16136
